In [1]:
import pvdeg
import matplotlib.pyplot as plt

## Implemented PVLIB temperature models in PVDEG 

In [3]:
cell_temp = pvdeg.scenario.Scenario(
    name= 'cell temp',
    api_key='DEMO_KEY',
    email='user@mail.com',
)

cell_temp.addLocation(
    lat_long=(25.783388, -80.189029), 
    see_added=True,
)

# irradiance args arent getting caried through, module args probably arent either

cell_temp.addModule(
    module_name='module_1',
    temperature_model='faiman'
)

cell_temp.addModule(
    module_name='module_1',
    racking='open_rack_glass_polymer',
    material='EVA',
    temperature_model='sapm',
    irradiance_kwarg={'azimuth' : 120, 'tilt' : 30}, # check if this changes anything
    model_kwarg={'irrad_ref' : 1100}, # check if this changes anything, if it doesnt then we have a problem
    see_added=True
)

cell_temp.addModule(
    module_name='module_2',
    racking='freestanding',
    material='EVA',
    temperature_model='pvsyst',
    irradiance_kwarg={'azimuth' : 180, 'tilt' : 0}, # check if this changes anything
    model_kwarg={'module_efficiency' : 0.15}, # check if this changes anything, if it doesnt then we have a problem
    see_added=True
)
cell_temp.addModule(
    module_name='module_3',
    racking='open_rack_glass_polymer',
    material='EVA',
    temperature_model='sapm',
    irradiance_kwarg={'azimuth' : 120, 'tilt' : 30}, # check if this changes anything
    model_kwarg={'irrad_ref' : 1000}, # check if this changes anything, if it doesnt then we have a problem
    see_added=True
)
cell_temp.addModule(
    module_name='module_4',
    racking='open_rack_glass_polymer',
    material='EVA',
    temperature_model='sapm',
    irradiance_kwarg={'azimuth' : 180, 'tilt' : 0}, # check if this changes anything
    model_kwarg={'irrad_ref' : 1000}, # check if this changes anything, if it doesnt then we have a problem
    see_added=True
)

cell_temp.updatePipeline(
    func=pvdeg.temperature.temperature,
    func_params={'cell_or_mod' : 'cell'}, # check this and change
    see_added=True
)

cell_temp.updatePipeline(
    # func=pvdeg.standards.standoff, # standoff only works with sapm, pvsyst breaks everything because of the configurations, default conf_inf of insulated_back_glass_polymer so if we try temp model = pysyst we will try to access an invalid configuration for that model
    func=pvdeg.degradation.vantHoff_deg,
    func_params={'I_chamber' : 1000, 'temp_chamber' : 25},
    see_added=True
)

cell_temp.updatePipeline(
    func=pvdeg.degradation.IwaVantHoff,
    see_added=True
)

Column "relative_humidity" not found in DataFrame. Calculating...
WARNING - Module already found by name "module_1"
Module will be replaced with new instance.
Module "module_1" added.
Module "module_2" added.
Module "module_3" added.
Module "module_4" added.
dict_keys(['I_chamber', 'temp_chamber'])
dict_keys(['poa', 'temp'])
FAILED: Requestion function <function vantHoff_deg at 0x000002B964492680> did not receive enough parameters
Requestion function: 
 <function vantHoff_deg at 0x000002B964492680> 
 ---
Required Parameters: 
 {'poa': <Parameter "poa=None">, 'temp': <Parameter "temp=None">} 
 ---
Optional Parameters: {'weather_df': <Parameter "weather_df">, 'meta': <Parameter "meta">, 'I_chamber': <Parameter "I_chamber">, 'temp_chamber': <Parameter "temp_chamber">, 'p': <Parameter "p=0.5">, 'Tf': <Parameter "Tf=1.41">, 'temp_model': <Parameter "temp_model='sapm'">, 'conf': <Parameter "conf='open_rack_glass_polymer'">, 'wind_factor': <Parameter "wind_factor=0.33">, 'irradiance_kwarg': <

C:\Users\tford\dev\PVDegradationTools\pvdeg\scenario.py:291: UserWarning: Gids Added - [1060699]
  warnings.warn(message, UserWarning)
C:\Users\tford\dev\PVDegradationTools\pvdeg\scenario.py:533: UserWarning: temperature added to pipeline as 
 {'job': <function temperature at 0x000002B963338D30>, 'params': {'cell_or_mod': 'cell'}}
  warnings.warn(message, UserWarning)
C:\Users\tford\dev\PVDegradationTools\pvdeg\scenario.py:533: UserWarning: IwaVantHoff added to pipeline as 
 {'job': <function IwaVantHoff at 0x000002B9644927A0>, 'params': {}}
  warnings.warn(message, UserWarning)


In [ ]:
cell_temp.runPipeline()

In [ ]:
cell_temp.clean()

In [ ]:
mod_1_temp = cell_temp.results['module_1']['temperature'][:24]

new_index = mod_1_temp.index.map(lambda dt: dt.replace(year=2000))

plt.plot(new_index, mod_1_temp)
plt.plot(new_index, cell_temp.results['module_2']['temperature'][:24])
plt.plot(new_index, cell_temp.results['module_3']['temperature'][:24])
plt.plot(new_index, cell_temp.results['module_4']['temperature'][:24])
plt.title('Single Day Cell (sapm) Temperature in $\degree$C')